# Data Catalog Beta Scenario 1: Analyze customer demographics data

In this notebook, you'll analyze customer demographics, such as, age, gender, income, and location. You'll load data from a data asset that originated in a governed catalog in Data Catalog as well as from a public open data set, cleanse, shape, and enrich the data, and then visualize the data. By the end of the notebook, you'll understand how to combine data to gain insights about which customers you might target to increase sales.

This notebook runs on Python 2 with Spark 2.0.


<a id="toc"></a>
## Table of contents

#### [Setup](#Setup)
[Load data into the notebook](#Load-data-into-the-notebook)
#### [Explore customer demographics](#part1)
[Prepare the customer data set](#Prepare-the-customer-data-set)<br>
[Visualize customer demographics and locations](#Visualize-customer-demographics-and-locations)<br>
[Enrich demographic information with open data](#Enrich-demographic-information-with-open-data)<br>   
#### [Summary](#summary)

## Setup
You need to import libraries and load the customer data into this notebook.

Import the necessary libraries:

In [ ]:
import pixiedust
import pyspark.sql.functions as func
import pyspark.sql.types as types
import re
import json
import os
import requests  

### Load data into the notebook

The data file you'll be loading contains the customer demographic data required for this exercise.

**Option 1: Load data from project data asset** 

If you are following the Data Manager Beta scenario, you should have a data asset called **dm-beta-scenario-customers.csv** in the project. To load this data into the notebook:
* Click on the the empty input line **In [ ]** below so that your cursor is in the input line.
* Click on the **"1001" icon** (Find and Add Data) located at the top right corner of the toolbar. 
* You should see the data asset **dm-beta-scenario-customers.csv** listed under **Files**.
* Click on **Insert to code**.
* Select **Insert SparkSession DataFrame**.
* You should see lines of code being added to the input line. The first line should say **from pyspark.sql import SparkSession**.
* In the last four lines of code, ensure that the data is read into a data frame named **df_data_1**. Update if necessary, e.g. **df_data_1 = spark.read** and **df_data_1.take(5)**.

**Option 2: Load data from URL** 

If you do not have the data asset in the project, you can copy and paste the following line into input line below:

`df_data_1 = pixiedust.sampleData('https://raw.githubusercontent.com/wdp-beta/get-started/master/data/dm-beta-scenario-customers.csv')`

[Back to Table of Contents](#toc)
<a id="part1"></a>
# Explore customer demographics 
In this notebook, you'll prepare the customer data and then start learning about your customers by creating multiple charts and maps. 

## Prepare the customer data set
You'll create a new DataFrame with just the data you need and then cleanse and enrich the data.

Extract the columns that you want, remove duplicate customers, and add a column for aggregations:

In [ ]:
# Extract the customer information from the data set
# CUSTNAME: string, GenderCode: string, ADDRESS1: string, CITY: string, STATE: string, COUNTRY_CODE: string, POSTAL_CODE: string, POSTAL_CODE_PLUS4: int, ADDRESS2: string, EMAIL_ADDRESS: string, PHONE_NUMBER: string, CREDITCARD_TYPE: string, LOCALITY: string, SALESMAN_ID: string, NATIONALITY: string, NATIONAL_ID: string, CREDITCARD_NUMBER: bigint, DRIVER_LICENSE: string, CUST_ID: int,
customer_df = df_data_1.select("CUST_ID", 
                            "CUSTNAME", 
                            "ADDRESS1", 
                            "ADDRESS2", 
                            "CITY", 
                            "POSTAL_CODE", 
                            "POSTAL_CODE_PLUS4", 
                            "STATE", 
                            "COUNTRY_CODE", 
                            "EMAIL_ADDRESS", 
                            "PHONE_NUMBER",
                            "AGE",
                            "GenderCode",
                            "GENERATION",
                            "NATIONALITY", 
                            "NATIONAL_ID", 
                            "DRIVER_LICENSE").dropDuplicates()

# append a column to the DataFrame for aggregations
customer_df = customer_df.withColumn("count", func.lit(1))
customer_df

Notice that the data type of the AGE column is currently a string. Convert the AGE column to a numeric data type so you can run calculations on customer age.

In [ ]:
# ---------------------------------------
# Cleanse age (enforce numeric data type) 
# ---------------------------------------

def getNumericVal(col):
    """
    input: pyspark.sql.types.Column
    output: the numeric value represented by col or None
    """
    try:
      return int(col)
    except ValueError:
      # age-33
      match = re.match('^age\-(\d+)$', col)
      if match:
        try:
          return int(match.group(1))
        except ValueError:    
          return None
      return None  

toNumericValUDF = func.udf(lambda c: getNumericVal(c), types.IntegerType())
customer_df = customer_df.withColumn("AGE", toNumericValUDF(customer_df["AGE"]))

The GenderCode column contains salutations instead of gender values. Derive the gender information for each customer based on the salutation and rename the GenderCode column to GENDER.

In [ ]:
# ------------------------------
# Derive gender from salutation
# ------------------------------
def deriveGender(col):
    """ input: pyspark.sql.types.Column
        output: "male", "female" or "unknown"
    """    
    if col in ['Mr.', 'Master.']:
        return 'male'
    elif col in ['Mrs.', 'Miss.']:
        return 'female'
    else:
        return 'unknown';
    
deriveGenderUDF = func.udf(lambda c: deriveGender(c), types.StringType())
customer_df = customer_df.withColumn("GENDER", deriveGenderUDF(customer_df["GenderCode"]))
customer_df.cache()

[Back to Table of Contents](#toc)
## Visualize customer demographics and locations

Now you're ready explore the customer base. Using simple charts, you can quickly see these characteristics:
 * Customer demographics (gender and age)
 * Customer locations (city, state, and country)

You'll create charts with the PixieDust library:

 - [View customers by gender in a pie chart](#View-customers-by-gender-in-a-pie-chart)
 - [View customers by generation in a bar chart](#View-customers-by-generation-in-a-bar-chart)
 - [View customers by age in a histogram chart](#View-customers-by-age-in-a-histogram-chart)
 - [View specific information with a filter function](#View-specific-information-with-a-filter-function)
 - [View customer density by location with a map](#View-customer-density-by-location-with-a-map)

### View customers by gender in a pie chart

Run the `display()` command and then configure the graph to show the percentages of male and female customers:

1. Run the next cell. The PixieDust interactive widget appears.  
1. Click the chart button and choose **Pie Chart**. The chart options tool appears.
1. In the chart options tool, drag `count` into the **Values** box. 
1. Move `GENDER` into the **Keys** box. 
1. In the **Aggregation** field, choose **COUNT**. 
1. Click **OK**. The pie chart appears.

If you want to make further changes, click **Options** to return to the chart options tool.

In [ ]:
display(customer_df)

[Back to Table of Contents](#toc)
### View customers by generation in a bar chart
Look at how many customers you have per "generation."

Run the next cell and configure the graph: 
1. Choose **Bar Chart** as the chart type.
2. Put `GENERATION` into the **Keys** box.
3. Put `count` into the **Values** box.
4. Set aggregation to COUNT.

In [ ]:
display(customer_df)

You can use clustering to group customers, for example by geographic location. To group generations by country, select `COUNTRY_CODE` from the **Cluster by** list. 

[Back to Table of Contents](#toc)
### View customers by age in a histogram chart
A generation is a broad age range. You can look at a smaller age range with a histogram chart. A histogram is like a bar chart except each bar represents a range of numbers, called a bin. You can customize the size of the age range by adjusting the bin size. The more bins you specify, the smaller the age range.

Run the next cell and configure the graph:
1. Choose **Histogram** as the chart type.
2. Put `AGE` into the **Keys** box and click **OK**.
3. Use the **Bin size** slider to specify the number of the bins. Try starting with 40.

In [ ]:
display(customer_df)

[Back to Table of Contents](#toc)
### View specific information with a filter function

You can filter records to restrict analysis by using the `filter()` function.

If you want to view the age distribution for a specific generation, uncomment and run one of the following commands:

Run the next cell and configure the graph:
1. Choose **Histogram** as the chart type.
2. Put `AGE` into the **Keys** box and click **OK**.


In [ ]:
# Data subsetting: display age distribution for a specific generation
display(customer_df.filter("GENERATION = 'Baby_Boomers'"))
#display(customer_df.filter("GENERATION = 'Gen_X'"))
#display(customer_df.filter("GENERATION = 'Gen_Y'"))
#display(customer_df.filter("GENERATION = 'Gen_Z'"))

You can also filter by location. For example, the following command creates a new DataFrame that filters for customers from the USA:

In [ ]:
us_customer_df = customer_df.filter("COUNTRY_CODE = 'US'")

You can pivot your analysis perspective based on aspects that are of interest to you by choosing different keys and clusters.

Create a bar chart and cluster the data.

Run the next cell and configure the graph:
1. Choose **Bar chart** as the chart type.
2. Put `COUNTRY_CODE` into the **Keys** box.
3. Put `count` in the **Values** box.
4. Set Aggregation to **COUNT**.
5. Click **OK**. The chart displays the number of US customers.
6. From the **Cluster By** list, choose **GENDER**. The chart shows the number of customers by gender.

In [ ]:
display(us_customer_df)

Now try to cluster the customers by state.

A bar chart isn't the best way to show geographic location!

[Back to Table of Contents](#toc)
### View customer density by location with a map
Maps are a much better way to view location data than other chart types. 

Visualize customer density by US state with a map.

Run the next cell and configure the graph:
1. Choose **Map** as the chart type.
2. Put `STATE` into the **Keys** box.
3. Put `count` in the **Values** box.
4. Set Aggregation to **COUNT**.
5. Click **OK**. The map displays the number of US customers.
6. From the **Renderer** list, choose **google**.
7. From the **Display Mode** list, choose **region**.
8. From the **Region** list, choose `US`.

In [ ]:
display(us_customer_df)

Hover over an area to display the number of customers in each state.

You can explore more about customers in each state by changing the aggregation method. 

Look at customer age ranges (avg, minimum, and maximum) by state.

Run the next cell and configure the graph:
1. Choose **Map** as the chart type.
2. Put `STATE` into the **Keys** box.
3. Put `AGE` in the **Values** box.
4. Set Aggregation to **AVG**, **MAX**, or **MIN**.
5. Click **OK**. The map displays.
6. From the **Renderer** list, choose **google**.
7. From the **Display Mode** list, choose **region**.
8. From the **Region** list, choose `US`.

In [ ]:
display(us_customer_df)

[Back to Table of Contents](#toc)
## Enrich demographic information with open data
You can easily combine other sources of data with your existing data. There's a lot of publicly available open data sets that can be very helpful. For example, knowing the approximate income level of your customers might help you target your marketing campaigns.

Run the next cell to load [this data set](https://apsportal.ibm.com/exchange/public/entry/view/beb8c30a3f559e58716d983671b70337) from the **United States Census Bureau** into your notebook. The data set contains US household income statistics compiled at the zip code geography level.

In [ ]:
# Load median income information for all US ZIP codes from a public source
income_df = pixiedust.sampleData('https://apsportal.ibm.com/exchange-api/v1/entries/beb8c30a3f559e58716d983671b70337/data?accessKey=1c0b5b6d465fefec1ab529fde04997af')

Now cleanse the income data set to remove the data that you don't need. Create a new DataFrame for this data:
 - The zip code, extracted from the GEOID column.
 - The column B19049e1, which contains the median household income for 2013.

In [ ]:
# ------------------------------
# Helper: Extract ZIP code
# ------------------------------
def extractZIPCode(col):
    """ input: pyspark.sql.types.Column containing a geo code, like '86000US01001'
        output: ZIP code
    """
    m = re.match('^\d+US(\d\d\d\d\d)$',col)
    if m:
        return m.group(1)
    else:
        return None    
    
getZIPCodeUDF = func.udf(lambda c: extractZIPCode(c), types.StringType())
income_df = income_df.select('GEOID', 'B19049e1').withColumnRenamed('B19049e1', 'MEDIAN_INCOME_IN_ZIP').withColumn("ZIP", getZIPCodeUDF(income_df['GEOID']))
income_df

Now perform a left outer join on the customer data set with the income data set, using the zip code as the join condition. For the complete syntax of joins, go to the <a href="https://spark.apache.org/docs/1.5.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame" target="_blank" rel="noopener noreferrer">pyspark DataFrame documentation</a> and scroll down to the `join` syntax. 

In [ ]:
us_customer_df = us_customer_df.join(income_df, us_customer_df.POSTAL_CODE == income_df.ZIP, 'left_outer').drop('GEOID').drop('ZIP')

Now you can visualize the income distribution of your customers by zip code.
 Visualize income distribution for our customers.
Run the next cell and configure the graph:
1. Choose **Histogram** as the chart type.
2. Put `MEDIAN_INCOME_IN_ZIP` into the **Keys** box and click **OK**.
3. For **Renderer**, select **bokeh**.

In [ ]:
display(us_customer_df)

The majority of your customers live in zip codes where the median income is around 40,000 USD. 

[Back to Table of Contents](#toc)
<a id="summary"></a>
## Summary and next steps

You've learned how to cleanse, shape, enrich, and visualize customer and sales data to find valuable insights. 

### Authors
Patrick Titzler is a customer advocate for Watson Data Platform at IBM.

<hr>
Copyright &copy; IBM Corp. 2017. This notebook and its source code are released under the terms of the MIT License.